In [16]:
import pandas as pd

dados = pd.read_csv('./manutencao_preditiva.csv')
dados

,UDI,ID Produto,Tipo,Temperatura Ar [K],Temperatura Processo [K],Velocidade Rotacao [rpm],Torque [Nm],Desgaste Ferramenta [min],Alvo,Tipo da Falha
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [17]:
def padronizar_tipo(valor):
    if valor == 'L':
        return 0.0
    elif valor == 'M':
        return 0.5
    elif valor == 'H':
        return 1.0
    else:
        return ValueError

In [18]:
def normalziar_coluna(var1):
    normalizado = []
    for i in range(len(var1)):
        normalizado.append((var1[i] - min(var1)) / (max(var1) - min(var1)))
    return normalizado

In [19]:
def delta_entre_coluna(var1, var2):
    diferenca = []
    if len(var1) == len(var2):
        for i in range(len(var1)):
            diferenca.append( (abs(var1[i] - var2[i])) )
        return diferenca
    else:
        return ValueError
        

In [20]:
def media_da_coluna(x):
    return (sum(x)/len(x))

In [21]:
def padronizar_falhas(valor):
    if valor == 'No Failure':
        return 0
    if valor == 'Heat Dissipation Failure':
        return 1
    elif valor == 'Power Failure':
        return 10
    elif valor == 'Overstrain Failure':
        return 100
    elif valor == 'Tool Wear Failure':
        return 1000
    elif valor == 'Random Failures':
        return 10000
    else:
        return ValueError

In [22]:
dados['Tipo'].value_counts()

Tipo
L    6000
M    2997
H    1003
Name: count, dtype: int64

In [23]:
falhas = dados['Tipo da Falha'].value_counts()
falhas

Tipo da Falha
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64

In [24]:
int(falhas['No Failure'])

9652

In [25]:
dados['Alvo'].value_counts()

Alvo
0    9661
1     339
Name: count, dtype: int64

In [26]:
dados['Falhas Padronizadas'] = dados['Tipo da Falha'].apply(padronizar_falhas)

In [27]:
dados['Temp Ar Normalizado'] = normalziar_coluna(dados['Temperatura Ar [K]'])
dados['Temp Processo Normalizado'] = normalziar_coluna(dados['Temperatura Processo [K]'])

In [28]:
dados['RPM Normalizado'] = normalziar_coluna(dados['Velocidade Rotacao [rpm]'])
dados['Torque Normalizado'] = normalziar_coluna(dados['Torque [Nm]'])
dados['Desgaste Normalizado'] = normalziar_coluna(dados['Desgaste Ferramenta [min]'])

In [29]:
dados['Tipo padronizado'] = dados['Tipo'].apply(padronizar_tipo)


In [30]:
dados['Delta Temp'] = delta_entre_coluna(dados['Temperatura Processo [K]'], dados['Temperatura Ar [K]'])
dados['Delta Temp Normalizado'] = normalziar_coluna(dados['Delta Temp'])


In [31]:
dados['Delta RPM-Torque Norm'] = delta_entre_coluna(dados['RPM Normalizado'], dados['Torque Normalizado'])
dados['Delta RPM-Desgaste Norm'] = delta_entre_coluna(dados['RPM Normalizado'], dados['Desgaste Normalizado'])
dados['Delta Desgaste-Torque Norm'] = delta_entre_coluna(dados['Desgaste Normalizado'], dados['Torque Normalizado'])

In [32]:
nova_ordem = ['Tipo', 'Tipo padronizado', 'Temperatura Ar [K]', 'Temp Ar Normalizado', 'Temperatura Processo [K]', 'Temp Processo Normalizado', 'Delta Temp', 'Delta Temp Normalizado', 'Velocidade Rotacao [rpm]', 'RPM Normalizado', 'Torque [Nm]', 'Torque Normalizado', 'Desgaste Ferramenta [min]', 'Desgaste Normalizado', 'Delta RPM-Torque Norm', 'Delta RPM-Desgaste Norm', 'Delta Desgaste-Torque Norm', 'Falhas Padronizadas','Tipo da Falha', 'Alvo']

In [38]:
sort_order = ['Tipo padronizado', 'Falhas Padronizadas']
sort_ascending = [True, True]

In [39]:
dados = dados[nova_ordem]
dados = dados.sort_values(by=sort_order, ascending=sort_ascending)
dados

,Tipo,Tipo padronizado,Temperatura Ar [K],Temp Ar Normalizado,Temperatura Processo [K],Temp Processo Normalizado,Delta Temp,Delta Temp Normalizado,Velocidade Rotacao [rpm],RPM Normalizado,Torque [Nm],Torque Normalizado,Desgaste Ferramenta [min],Desgaste Normalizado,Delta RPM-Torque Norm,Delta RPM-Desgaste Norm,Delta Desgaste-Torque Norm,Falhas Padronizadas,Tipo da Falha,Alvo
1,L,0.0,298.2,0.315217,308.7,0.370370,10.5,0.644444,1408,0.139697,46.3,0.583791,3,0.011858,0.444094,0.127840,0.571934,0,No Failure,0
2,L,0.0,298.1,0.304348,308.5,0.345679,10.4,0.622222,1498,0.192084,49.4,0.626374,5,0.019763,0.434290,0.172321,0.606611,0,No Failure,0
3,L,0.0,298.2,0.315217,308.6,0.358025,10.4,0.622222,1433,0.154249,39.5,0.490385,7,0.027668,0.336135,0.126581,0.462717,0,No Failure,0
4,L,0.0,298.2,0.315217,308.7,0.370370,10.5,0.644444,1408,0.139697,40.0,0.497253,9,0.035573,0.357555,0.104124,0.461680,0,No Failure,0
6,L,0.0,298.1,0.304348,308.6,0.358025,10.5,0.644444,1558,0.227008,42.4,0.530220,14,0.055336,0.303212,0.171672,0.474884,0,No Failure,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419,H,1.0,300.3,0.543478,309.9,0.518519,9.6,0.444444,1394,0.131548,46.7,0.589286,210,0.830040,0.457737,0.698491,0.240754,1000,Tool Wear Failure,1
1748,H,1.0,298.4,0.336957,307.7,0.246914,9.3,0.377778,1626,0.266589,31.1,0.375000,166,0.656126,0.108411,0.389537,0.281126,10000,Random Failures,0
3452,H,1.0,301.6,0.684783,310.5,0.592593,8.9,0.288889,1602,0.252619,32.3,0.391484,2,0.007905,0.138864,0.244714,0.383578,10000,Random Failures,0
5495,H,1.0,302.9,0.826087,312.5,0.839506,9.6,0.444444,1357,0.110012,55.0,0.703297,12,0.047431,0.593285,0.062581,0.655866,10000,Random Failures,0


In [34]:
colunas = dados.columns

print("Valores originais:")
for col in colunas[3:8]:
    print(f"Media da coluna - {col} é: {media_da_coluna(dados[col]):.02f}")

print("-------------------\nValores normalizados: ")
for col in colunas[10:16]:
    print(f"Media da coluna - {col} é: {media_da_coluna(dados[col]):.02f}")

print("-------------------\nValores de Deltas: ")
for col in colunas[16:]:
    print(f"Media da coluna - {col} é: {media_da_coluna(dados[col]):.02f}")

Valores originais:
Media da coluna - Temp Ar Normalizado é: 0.51
Media da coluna - Temperatura Processo [K] é: 310.01
Media da coluna - Temp Processo Normalizado é: 0.53
Media da coluna - Delta Temp é: 10.00
Media da coluna - Delta Temp Normalizado é: 0.53
-------------------
Valores normalizados: 
Media da coluna - Torque [Nm] é: 39.99
Media da coluna - Torque Normalizado é: 0.50
Media da coluna - Desgaste Ferramenta [min] é: 107.95
Media da coluna - Desgaste Normalizado é: 0.43
Media da coluna - Delta RPM-Torque Norm é: 0.32
Media da coluna - Delta RPM-Desgaste Norm é: 0.28
-------------------
Valores de Deltas: 
Media da coluna - Delta Desgaste-Torque Norm é: 0.25
Media da coluna - Falhas Padronizadas é: 23.39


TypeError: unsupported operand type(s) for +: 'int' and 'str'